# Wczytanie danych oraz wstępny wybór

In [37]:
import pandas as pd
import numpy as np
pd.options.display.max_columns = 60
pd.options.display.max_info_columns = 60
pd.options.display.max_rows = 100

data_source = pd.read_csv(r"data_2022/seasons_stats.csv",encoding='latin-1')
cols = ['Tm','Year', 'Player', 'Pos', 'Age', 'G', 'GS', 'MP',
       'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%',
       'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV',
       'PF', 'PTS', 'PER', 'TS%', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%',
       'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'OWS', 'DWS', 'WS', 'WS/48',
       'OBPM', 'DBPM', 'BPM', 'VORP']       
data_source = data_source[cols]
#data_source = data_source.query("Year < 1982").reset_index(drop=True)
#data_source = data_source.query("Year >= 1982 and Year < 2022").reset_index(drop=True)
data_source.info() #20935 rows

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28057 entries, 0 to 28056
Data columns (total 50 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Tm      28057 non-null  object 
 1   Year    28057 non-null  int64  
 2   Player  28057 non-null  object 
 3   Pos     28057 non-null  object 
 4   Age     28049 non-null  float64
 5   G       28057 non-null  int64  
 6   GS      21666 non-null  float64
 7   MP      27571 non-null  float64
 8   FG      28057 non-null  int64  
 9   FGA     28057 non-null  int64  
 10  FG%     27942 non-null  float64
 11  3P      22360 non-null  float64
 12  3PA     22360 non-null  float64
 13  3P%     18632 non-null  float64
 14  2P      28057 non-null  int64  
 15  2PA     28057 non-null  int64  
 16  2P%     27878 non-null  float64
 17  eFG%    27942 non-null  float64
 18  FT      28057 non-null  int64  
 19  FTA     28057 non-null  int64  
 20  FT%     27007 non-null  float64
 21  ORB     24230 non-null  float64
 22

In [39]:
data = data_source
data[data['USG%'].isna()]

,Tm,Year,Player,Pos,Age,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP
0,FTW,1950,Curly Armstrong,G-F,31.0,63,NaN,NaN,144,516,0.279,NaN,NaN,NaN,144,516,0.279,0.279,170,241,0.705,NaN,NaN,NaN,176,NaN,NaN,NaN,217,458,NaN,0.368,NaN,0.467,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.1,3.6,3.5,NaN,NaN,NaN,NaN,NaN
1,INO,1950,Cliff Barker,SG,29.0,49,NaN,NaN,102,274,0.372,NaN,NaN,NaN,102,274,0.372,0.372,75,106,0.708,NaN,NaN,NaN,109,NaN,NaN,NaN,99,279,NaN,0.435,NaN,0.387,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.6,0.6,2.2,NaN,NaN,NaN,NaN,NaN
2,CHS,1950,Leo Barnhorst,SF,25.0,67,NaN,NaN,174,499,0.349,NaN,NaN,NaN,174,499,0.349,0.349,90,129,0.698,NaN,NaN,NaN,140,NaN,NaN,NaN,192,438,NaN,0.394,NaN,0.259,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.9,2.8,3.6,NaN,NaN,NaN,NaN,NaN
3,TOT,1950,Ed Bartels,F,24.0,15,NaN,NaN,22,86,0.256,NaN,NaN,NaN,22,86,0.256,0.256,19,34,0.559,NaN,NaN,NaN,20,NaN,NaN,NaN,29,63,NaN,0.312,NaN,0.395,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.5,-0.1,-0.6,NaN,NaN,NaN,NaN,NaN
4,DNN,1950,Ed Bartels,F,24.0,13,NaN,NaN,21,82,0.256,NaN,NaN,NaN,21,82,0.256,0.256,17,31,0.548,NaN,NaN,NaN,20,NaN,NaN,NaN,27,59,NaN,0.308,NaN,0.378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.5,-0.1,-0.6,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17731,HOU,2006,Josh Davis,PF,25.0,1,0.0,0.0,0,0,NaN,0.0,0.0,NaN,0,0,NaN,NaN,0,0,NaN,0.0,0.0,0.0,0,0.0,0.0,0.0,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,-7.2,0.9,-6.3,0.0
18048,SAS,2006,Alex Scales,SG,27.0,1,0.0,0.0,0,0,NaN,0.0,0.0,NaN,0,0,NaN,NaN,0,0,NaN,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,-6.0,1.6,-4.4,0.0
19011,GSW,2008,Stephane Lasme,SF,25.0,1,0.0,0.0,0,0,NaN,0.0,0.0,NaN,0,0,NaN,NaN,0,0,NaN,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,-6.0,-0.4,-6.4,0.0
19981,LAC,2010,JamesOn Curry,PG,24.0,1,0.0,0.0,0,0,NaN,0.0,0.0,NaN,0,0,NaN,NaN,0,0,NaN,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,-6.7,-0.2,-6.9,0.0


In [5]:
data_source[data_source.duplicated(subset = ['Year', 'Player'],keep=False)][15:31]

,Tm,Year,Player,Pos,Age,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP
65,TOT,1982,Charlie Criss,PG,33.0,55,20.0,1392.0,222,498,0.446,10.0,29.0,0.345,212,469,0.452,0.456,141,159,0.887,13.0,69.0,82.0,187,44.0,6.0,82.0,96,595,13.3,0.524,0.058,0.319,1.1,5.9,3.4,19.2,1.5,0.3,12.6,19.5,1.6,0.4,2.1,0.072,-0.5,-2.5,-3.0,-0.4
66,ATL,1982,Charlie Criss,PG,33.0,27,0.0,552.0,84,210,0.400,2.0,8.0,0.250,82,202,0.406,0.405,65,73,0.890,6.0,32.0,38.0,75,23.0,2.0,30.0,40,235,13.6,0.485,0.038,0.348,1.2,6.6,3.9,20.7,2.1,0.2,11.0,21.4,0.4,0.6,1.0,0.089,-1.2,-1.3,-2.5,-0.1
67,SDC,1982,Charlie Criss,PG,33.0,28,20.0,840.0,138,288,0.479,8.0,21.0,0.381,130,267,0.487,0.493,76,86,0.884,7.0,37.0,44.0,112,21.0,4.0,52.0,56,360,13.1,0.552,0.073,0.299,1.0,5.4,3.1,18.1,1.2,0.3,13.8,18.3,1.2,-0.2,1.1,0.061,-0.1,-3.3,-3.4,-0.3
98,TOT,1982,Mike Evans,PG,26.0,22,0.0,270.0,35,86,0.407,0.0,6.0,0.000,35,80,0.438,0.407,13,20,0.650,5.0,17.0,22.0,42,13.0,0.0,26.0,36,83,7.3,0.438,0.070,0.233,2.1,7.2,4.6,21.0,2.3,0.0,21.5,18.4,-0.4,0.3,-0.1,-0.013,-4.4,-1.1,-5.5,-0.2
99,MIL,1982,Mike Evans,PG,26.0,14,0.0,196.0,24,51,0.471,0.0,2.0,0.000,24,49,0.490,0.471,8,12,0.667,3.0,9.0,12.0,22,9.0,0.0,15.0,26,56,7.2,0.498,0.039,0.235,1.8,5.1,3.5,14.5,2.2,0.0,21.0,15.0,-0.1,0.2,0.2,0.041,-4.0,-0.8,-4.8,-0.1
100,CLE,1982,Mike Evans,PG,26.0,8,0.0,74.0,11,35,0.314,0.0,4.0,0.000,11,31,0.355,0.314,5,8,0.625,2.0,8.0,10.0,20,4.0,0.0,11.0,10,27,7.5,0.350,0.114,0.229,2.9,13.0,7.6,38.0,2.6,0.0,22.2,27.6,-0.3,0.0,-0.2,-0.158,-5.4,-1.8,-7.2,-0.1
129,TOT,1982,Steve Hayes,C,26.0,35,0.0,487.0,54,111,0.486,0.0,0.0,NaN,54,111,0.486,0.486,32,53,0.604,39.0,78.0,117.0,28,4.0,20.0,19.0,71,140,10.7,0.521,0.000,0.477,8.5,18.0,13.1,7.3,0.4,2.3,12.4,12.3,0.5,0.5,0.9,0.092,-2.1,0.2,-1.9,0.0
130,SAS,1982,Steve Hayes,C,26.0,9,0.0,75.0,8,18,0.444,0.0,0.0,NaN,8,18,0.444,0.444,7,12,0.583,7.0,10.0,17.0,4,1.0,2.0,2.0,17,23,9.7,0.494,0.000,0.667,10.0,14.3,12.1,6.4,0.6,1.5,7.9,13.4,0.1,0.1,0.2,0.099,-1.5,-0.8,-2.3,0.0
131,DET,1982,Steve Hayes,C,26.0,26,0.0,412.0,46,93,0.495,0.0,0.0,NaN,46,93,0.495,0.495,25,41,0.610,32.0,68.0,100.0,24,3.0,18.0,17.0,54,117,10.9,0.527,0.000,0.441,8.3,18.7,13.3,7.4,0.3,2.4,13.3,12.1,0.4,0.4,0.8,0.091,-2.2,0.4,-1.8,0.0
137,TOT,1982,Armond Hill,PG,28.0,40,18.0,723.0,53,126,0.421,0.0,2.0,0.000,53,124,0.427,0.421,39,55,0.709,12.0,40.0,52.0,106,21.0,5.0,66.0,88,145,5.5,0.483,0.016,0.437,1.9,6.3,4.1,17.8,1.4,0.4,30.5,12.2,-0.6,0.2,-0.4,-0.024,-4.6,-1.3,-5.9,-0.7


In [38]:
missing_cels = data_source.isnull().sum().sum()
all_cels = data_source.shape[0] * data_source.shape[1]
print("Missing values:", round(missing_cels / all_cels * 100,4),"%")


Missing values: 7.3237 %


In [7]:
data_2022 = pd.read_csv(r"data/data_2022/totals_2022.csv",encoding='latin-1')
advanced_2022 = pd.read_csv(r"data/data_2022/advanced_2022.csv",encoding='latin-1')

data_2022 = pd.concat([data_2022,advanced_2022],axis=1)
cols_2022 = ['Player', 'Pos', 'Age','G', 'GS', 'MP', 'FG', 'FGA', 'FG%',
       '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%',
       'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS',
       'PER', 'TS%', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%',
       'BLK%', 'TOV%', 'USG%','OWS', 'DWS', 'WS', 'WS/48',
       'OBPM', 'DBPM', 'BPM', 'VORP']
data_2022 = data_2022[cols_2022]
data_2022 = data_2022.loc[:,~data_2022.columns.duplicated()].copy()
data_2022.insert(0,'Year',2022)
data_2022.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 812 entries, 0 to 811
Data columns (total 49 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Year    812 non-null    int64  
 1   Player  812 non-null    object 
 2   Pos     812 non-null    object 
 3   Age     812 non-null    int64  
 4   G       812 non-null    int64  
 5   GS      812 non-null    int64  
 6   MP      812 non-null    int64  
 7   FG      812 non-null    int64  
 8   FGA     812 non-null    int64  
 9   FG%     797 non-null    float64
 10  3P      812 non-null    int64  
 11  3PA     812 non-null    int64  
 12  3P%     740 non-null    float64
 13  2P      812 non-null    int64  
 14  2PA     812 non-null    int64  
 15  2P%     784 non-null    float64
 16  eFG%    797 non-null    float64
 17  FT      812 non-null    int64  
 18  FTA     812 non-null    int64  
 19  FT%     715 non-null    float64
 20  ORB     812 non-null    int64  
 21  DRB     812 non-null    int64  
 22  TR

In [8]:
data_source = pd.concat([data_source,data_2022])
data_source.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21747 entries, 0 to 811
Data columns (total 50 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Tm      20935 non-null  object 
 1   Year    21747 non-null  int64  
 2   Player  21747 non-null  object 
 3   Pos     21747 non-null  object 
 4   Age     21747 non-null  float64
 5   G       21747 non-null  int64  
 6   GS      21747 non-null  float64
 7   MP      21747 non-null  float64
 8   FG      21747 non-null  int64  
 9   FGA     21747 non-null  int64  
 10  FG%     21631 non-null  float64
 11  3P      21747 non-null  float64
 12  3PA     21747 non-null  float64
 13  3P%     18155 non-null  float64
 14  2P      21747 non-null  int64  
 15  2PA     21747 non-null  int64  
 16  2P%     21558 non-null  float64
 17  eFG%    21631 non-null  float64
 18  FT      21747 non-null  int64  
 19  FTA     21747 non-null  int64  
 20  FT%     20760 non-null  float64
 21  ORB     21747 non-null  float64
 22  

In [9]:
data_source[data_source.duplicated(subset = ['Year', 'Player'],keep=False)][15:31]

,Tm,Year,Player,Pos,Age,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP
65,TOT,1982,Charlie Criss,PG,33.0,55,20.0,1392.0,222,498,0.446,10.0,29.0,0.345,212,469,0.452,0.456,141,159,0.887,13.0,69.0,82.0,187,44.0,6.0,82.0,96,595,13.3,0.524,0.058,0.319,1.1,5.9,3.4,19.2,1.5,0.3,12.6,19.5,1.6,0.4,2.1,0.072,-0.5,-2.5,-3.0,-0.4
66,ATL,1982,Charlie Criss,PG,33.0,27,0.0,552.0,84,210,0.400,2.0,8.0,0.250,82,202,0.406,0.405,65,73,0.890,6.0,32.0,38.0,75,23.0,2.0,30.0,40,235,13.6,0.485,0.038,0.348,1.2,6.6,3.9,20.7,2.1,0.2,11.0,21.4,0.4,0.6,1.0,0.089,-1.2,-1.3,-2.5,-0.1
67,SDC,1982,Charlie Criss,PG,33.0,28,20.0,840.0,138,288,0.479,8.0,21.0,0.381,130,267,0.487,0.493,76,86,0.884,7.0,37.0,44.0,112,21.0,4.0,52.0,56,360,13.1,0.552,0.073,0.299,1.0,5.4,3.1,18.1,1.2,0.3,13.8,18.3,1.2,-0.2,1.1,0.061,-0.1,-3.3,-3.4,-0.3
98,TOT,1982,Mike Evans,PG,26.0,22,0.0,270.0,35,86,0.407,0.0,6.0,0.000,35,80,0.438,0.407,13,20,0.650,5.0,17.0,22.0,42,13.0,0.0,26.0,36,83,7.3,0.438,0.070,0.233,2.1,7.2,4.6,21.0,2.3,0.0,21.5,18.4,-0.4,0.3,-0.1,-0.013,-4.4,-1.1,-5.5,-0.2
99,MIL,1982,Mike Evans,PG,26.0,14,0.0,196.0,24,51,0.471,0.0,2.0,0.000,24,49,0.490,0.471,8,12,0.667,3.0,9.0,12.0,22,9.0,0.0,15.0,26,56,7.2,0.498,0.039,0.235,1.8,5.1,3.5,14.5,2.2,0.0,21.0,15.0,-0.1,0.2,0.2,0.041,-4.0,-0.8,-4.8,-0.1
100,CLE,1982,Mike Evans,PG,26.0,8,0.0,74.0,11,35,0.314,0.0,4.0,0.000,11,31,0.355,0.314,5,8,0.625,2.0,8.0,10.0,20,4.0,0.0,11.0,10,27,7.5,0.350,0.114,0.229,2.9,13.0,7.6,38.0,2.6,0.0,22.2,27.6,-0.3,0.0,-0.2,-0.158,-5.4,-1.8,-7.2,-0.1
129,TOT,1982,Steve Hayes,C,26.0,35,0.0,487.0,54,111,0.486,0.0,0.0,NaN,54,111,0.486,0.486,32,53,0.604,39.0,78.0,117.0,28,4.0,20.0,19.0,71,140,10.7,0.521,0.000,0.477,8.5,18.0,13.1,7.3,0.4,2.3,12.4,12.3,0.5,0.5,0.9,0.092,-2.1,0.2,-1.9,0.0
130,SAS,1982,Steve Hayes,C,26.0,9,0.0,75.0,8,18,0.444,0.0,0.0,NaN,8,18,0.444,0.444,7,12,0.583,7.0,10.0,17.0,4,1.0,2.0,2.0,17,23,9.7,0.494,0.000,0.667,10.0,14.3,12.1,6.4,0.6,1.5,7.9,13.4,0.1,0.1,0.2,0.099,-1.5,-0.8,-2.3,0.0
131,DET,1982,Steve Hayes,C,26.0,26,0.0,412.0,46,93,0.495,0.0,0.0,NaN,46,93,0.495,0.495,25,41,0.610,32.0,68.0,100.0,24,3.0,18.0,17.0,54,117,10.9,0.527,0.000,0.441,8.3,18.7,13.3,7.4,0.3,2.4,13.3,12.1,0.4,0.4,0.8,0.091,-2.2,0.4,-1.8,0.0
137,TOT,1982,Armond Hill,PG,28.0,40,18.0,723.0,53,126,0.421,0.0,2.0,0.000,53,124,0.427,0.421,39,55,0.709,12.0,40.0,52.0,106,21.0,5.0,66.0,88,145,5.5,0.483,0.016,0.437,1.9,6.3,4.1,17.8,1.4,0.4,30.5,12.2,-0.6,0.2,-0.4,-0.024,-4.6,-1.3,-5.9,-0.7


# Usunięcie duplikatów

In [10]:
data = data_source.copy()
#Drop rows with all missing values 
data.dropna(axis=0,how='all',inplace=True)
#Check for duplicated rows
data.duplicated().any() #False
data.duplicated(subset = ['Year', 'Player']).any() #True 
data = data[~data.duplicated(subset = ['Year', 'Player'], keep = 'first')]

# Dodanie kolumny MVP

In [11]:
mvp_winners_data = pd.read_excel(r"data/mvp_winners.xlsx",header=None,names=['Year','Player'],engine='openpyxl')
mvp_winners_data['Year'] = (mvp_winners_data['Year'].str[:2] + mvp_winners_data['Year'].str[5:]).astype('double')
mvp_winners_data['MVP'] = 1
mvp_winners_data

data['Player'] = data['Player'].str.replace('*','')
data = pd.merge(data,mvp_winners_data,on=['Year','Player'],how='left')
data.loc[(data['Year'] == 2021 ) & ( data["Player"].str.find('Nikola Joki') != -1 ),'MVP'] = 1
data.loc[(data['Year'] == 2022 ) & ( data["Player"].str.find('Nikola Joki') != -1 ),'MVP'] = 1
data['MVP'] = data['MVP'].fillna(0)


# Wypełnienie brakujących wartości 

In [12]:
missing_cels = data.isnull().sum().sum()
all_cels = data.shape[0] * data.shape[1]
print("Missing values:", round(missing_cels / all_cels * 100,4),"%")


Missing values: 0.4752 %


In [13]:
data[data['USG%'].isna()]

,Tm,Year,Player,Pos,Age,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,MVP
9710,SAS,2006,Alex Scales,SG,27.0,1,0.0,0.0,0,0,NaN,0.0,0.0,NaN,0,0,NaN,NaN,0,0,NaN,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,-6.0,1.6,-4.4,0.0,0.0
11254,LAC,2010,JamesOn Curry,PG,24.0,1,0.0,0.0,0,0,NaN,0.0,0.0,NaN,0,0,NaN,NaN,0,0,NaN,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,-6.7,-0.2,-6.9,0.0,0.0
12738,BRK,2013,Damion James,SF,25.0,2,0.0,0.0,0,0,NaN,0.0,0.0,NaN,0,0,NaN,NaN,0,0,NaN,0.0,1.0,1.0,0,0.0,0.0,0.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,-5.5,0.3,-5.2,0.0,0.0


In [14]:
data.isnull().mean()
#Deleting 3 players with NAN that are not significant 
data = data[data['USG%'].notna()]
#Deleting players that didnt make single shot
data['FG%'].fillna(0,inplace=True)
data['2P%'].fillna(0,inplace=True)
data['3P%'].fillna(0,inplace=True)
data['eFG%'].fillna(0,inplace=True)
data['FT%'].fillna(0,inplace=True)
data['TS%'].fillna(0,inplace=True)
data['3PAr'].fillna(0,inplace=True)
data['FTr'].fillna(0,inplace=True)
data['TOV%'].fillna(0,inplace=True)
data.isnull().mean()


Tm        0.034235
Year      0.000000
Player    0.000000
Pos       0.000000
Age       0.000000
G         0.000000
GS        0.000000
MP        0.000000
FG        0.000000
FGA       0.000000
FG%       0.000000
3P        0.000000
3PA       0.000000
3P%       0.000000
2P        0.000000
2PA       0.000000
2P%       0.000000
eFG%      0.000000
FT        0.000000
FTA       0.000000
FT%       0.000000
ORB       0.000000
DRB       0.000000
TRB       0.000000
AST       0.000000
STL       0.000000
BLK       0.000000
TOV       0.000000
PF        0.000000
PTS       0.000000
PER       0.000000
TS%       0.000000
3PAr      0.000000
FTr       0.000000
ORB%      0.000000
DRB%      0.000000
TRB%      0.000000
AST%      0.000000
STL%      0.000000
BLK%      0.000000
TOV%      0.000000
USG%      0.000000
OWS       0.000000
DWS       0.000000
WS        0.000000
WS/48     0.000000
OBPM      0.000000
DBPM      0.000000
BPM       0.000000
VORP      0.000000
MVP       0.000000
dtype: float64

In [15]:
xd = pd.read_csv(r"data/votings/mvp_votings.csv")
xd = xd[["season","player","votes_first","points_won","points_max","award_share"]]
xd["season"] = (xd['season'].str[:2] + xd['season'].str[5:]).astype('double') 
xd = xd[xd["season"] > 1981]
xd = xd.reset_index(drop=True)
xd = xd[["player","votes_first","points_won","points_max","award_share","season"]]
xd.rename(columns={"player": "Player", "season": "Year"},inplace = True)
xd.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 590 entries, 0 to 589
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Player       590 non-null    object 
 1   votes_first  590 non-null    float64
 2   points_won   590 non-null    float64
 3   points_max   590 non-null    float64
 4   award_share  590 non-null    float64
 5   Year         590 non-null    float64
dtypes: float64(5), object(1)
memory usage: 27.8+ KB


In [16]:
votings_2000 = pd.read_table(r"data\votings\votings_2000.txt",usecols=["Player","First","Pts Won","Pts Max","Share"])
votings_2000.columns = ["Player","votes_first","points_won","points_max","award_share"]
votings_2000['Year'] = 2000

votings_2019 = pd.read_table(r"data\votings\votings_2019.txt",usecols=["Player","First","Pts Won","Pts Max","Share"])
votings_2019.columns = ["Player","votes_first","points_won","points_max","award_share"]
votings_2019['Year'] = 2019


votings_2020 = pd.read_table(r"data\votings\votings_2020.txt",usecols=["Player","First","Pts Won","Pts Max","Share"])
votings_2020.columns = ["Player","votes_first","points_won","points_max","award_share"]
votings_2020['Year'] = 2020

votings_2021 = pd.read_table(r"data\votings\votings_2021.txt",usecols=["Player","First","Pts Won","Pts Max","Share"])
votings_2021.columns = ["Player","votes_first","points_won","points_max","award_share"]
votings_2021['Year'] = 2021

votings_2022 = pd.read_table(r"data\votings\votings_2022.txt",usecols=["Player","First","Pts Won","Pts Max","Share"])
votings_2022.columns = ["Player","votes_first","points_won","points_max","award_share"]
votings_2022['Year'] = 2022

xd = pd.concat([xd,votings_2000,votings_2019,votings_2021,votings_2022,votings_2020])
xd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 657 entries, 0 to 11
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Player       657 non-null    object 
 1   votes_first  657 non-null    float64
 2   points_won   657 non-null    float64
 3   points_max   657 non-null    float64
 4   award_share  657 non-null    float64
 5   Year         657 non-null    float64
dtypes: float64(5), object(1)
memory usage: 35.9+ KB


In [17]:
data['Player'] = data['Player'].where(data['Player'] != "Nikola JokiÄ","Nikola Jokić" )
data['Player'] = data['Player'].where(data['Player'] != "Luka DonÄiÄ","Luka Dončić" )


In [18]:
xd['Player'] = xd['Player'].where(xd['Player'] != "World B. Free","World B." )
xd['Player'] = xd['Player'].where(xd['Player'] != "Micheal Ray Richardson","Micheal Ray" )
xd['Player'] = xd['Player'].where(xd['Player'] != "Joe Barry Carroll","Joe Barry" )
xd['Player'] = xd['Player'].where(xd['Player'] != "Joe Barry Carroll","Joe Barry" )

data_new = pd.merge(data,xd,on=['Year','Player'],how='left')
data_new.fillna(0,inplace=True)
data_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17672 entries, 0 to 17671
Data columns (total 55 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Tm           17672 non-null  object 
 1   Year         17672 non-null  int64  
 2   Player       17672 non-null  object 
 3   Pos          17672 non-null  object 
 4   Age          17672 non-null  float64
 5   G            17672 non-null  int64  
 6   GS           17672 non-null  float64
 7   MP           17672 non-null  float64
 8   FG           17672 non-null  int64  
 9   FGA          17672 non-null  int64  
 10  FG%          17672 non-null  float64
 11  3P           17672 non-null  float64
 12  3PA          17672 non-null  float64
 13  3P%          17672 non-null  float64
 14  2P           17672 non-null  int64  
 15  2PA          17672 non-null  int64  
 16  2P%          17672 non-null  float64
 17  eFG%         17672 non-null  float64
 18  FT           17672 non-null  int64  
 19  FTA 

In [19]:

data_new.describe()

,Year,Age,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,MVP,votes_first,points_won,points_max,award_share
count,17672.000000,17672.000000,17672.000000,17672.000000,17672.000000,17672.000000,17672.000000,17672.000000,17672.000000,17672.000000,17672.000000,17672.000000,17672.000000,17672.000000,17672.000000,17672.000000,17672.000000,17672.000000,17672.000000,17672.000000,17672.000000,17672.000000,17672.000000,17672.000000,17672.000000,17672.000000,17672.000000,17672.000000,17672.000000,17672.000000,17672.000000,17672.000000,17672.000000,17672.000000,17672.000000,17672.000000,17672.000000,17672.000000,17672.000000,17672.000000,17672.000000,17672.000000,17672.000000,17672.000000,17672.000000,17672.000000,17672.000000,17672.000000,17672.000000,17672.000000,17672.000000,17672.000000
mean,2003.800475,26.651143,53.485967,26.043572,1258.955976,201.954391,437.290346,0.440831,29.503508,83.650407,0.222214,172.450883,353.639939,0.465182,0.472786,100.258092,132.702241,0.703002,63.351177,159.403916,222.755093,120.420722,41.593651,26.252037,76.845801,114.883997,533.670383,12.775809,0.511397,0.193980,0.309418,6.004663,14.070801,10.038847,13.142202,1.634461,1.533001,14.400713,18.860055,1.373800,1.287387,2.662042,0.072343,-1.534840,-0.397629,-1.932385,0.636119,0.002320,0.248246,6.452744,39.182888,0.006031
std,11.652288,4.108003,25.281795,29.385690,915.041174,182.920863,382.808807,0.096682,45.380063,120.161668,0.180623,167.062753,332.229848,0.104691,0.099293,109.371716,139.191059,0.189167,66.305310,145.175991,204.720168,141.386983,37.705643,36.928903,66.641880,78.820808,487.911746,6.149832,0.095807,0.207824,0.215632,4.864932,6.515351,4.965558,9.469635,1.010381,1.756595,6.311463,5.455085,2.096676,1.225067,3.016844,0.100327,3.986079,2.175210,4.941934,1.394529,0.048112,4.194890,64.823243,202.760587,0.059318
min,1982.000000,18.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-90.600000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-3.300000,-1.000000,-2.100000,-2.519000,-73.800000,-31.100000,-86.700000,-2.600000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1994.000000,23.000000,34.000000,1.000000,406.750000,49.000000,115.000000,0.404000,0.000000,2.000000,0.000000,38.000000,84.000000,0.430000,0.441000,20.000000,28.000000,0.651750,15.000000,44.000000,62.000000,21.000000,11.000000,4.000000,22.000000,45.000000,127.000000,9.900000,0.481000,0.008000,0.192750,2.400000,9.300000,6.200000,6.500000,1.100000,0.400000,10.900000,15.300000,0.000000,0.300000,0.300000,0.038000,-3.200000,-1.500000,-3.800000,-0.200000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2004.000000,26.000000,61.000000,11.000000,1177.000000,156.000000,345.000000,0.445000,5.000000,21.000000,0.273000,123.000000,259.000000,0.473000,0.482000,64.000000,88.000000,0.748000,40.000000,126.000000,171.000000,72.000000,33.000000,13.000000,61.000000,112.000000,408.000000,12.900000,0.523000,0.116000,0.278000,5.000000,13.100000,9.200000,10.350000,1.500000,1.000000,13.650000,18.500000,0.600000,1.000000,1.700000,0.081000,-1.300000,-0.400000,-1.500000,0.100000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2014.000000,29.000000,76.000000,52.000000,2000.000000,309.000000,671.000000,0.488000,44.000000,129.000000,0.357000,258.000000,532.000000,0.512000,0.519000,144.000000,193.000000,0.813000,91.000000,230.000000,320.000000,166.000000,62.000000,33.000000,115.000000,174.000000,815.000000,15.900000,0.559000,0.345250,0.385000,8.800000,18.200000,13.400000,17.800000,2.000000,2.100000,17.000000,22.100000,2.200000,1.900000,4.100000,0.119000,0.400000,0.800000,0.600000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,2022.000000,44.000000

In [20]:
data_new[data_new['award_share'] > 0.02].head(10)

,Tm,Year,Player,Pos,Age,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,MVP,votes_first,points_won,points_max,award_share
0,LAL,1982,Kareem Abdul-Jabbar,C,34.0,76,76.0,2677.0,753,1301,0.579,0.0,3.0,0.000,753,1298,0.580,0.579,312,442,0.706,172.0,487.0,659.0,225,63.0,207.0,230.0,224,1818,23.4,0.608,0.002,0.340,7.3,19.1,13.4,11.9,1.1,4.1,13.3,25.6,6.9,3.9,10.7,0.192,2.9,1.4,4.3,4.2,0.0,0.0,31.0,690.0,0.045
14,BOS,1982,Larry Bird,PF,25.0,77,58.0,2923.0,711,1414,0.503,11.0,52.0,0.212,700,1362,0.514,0.507,328,380,0.863,200.0,637.0,837.0,447,143.0,66.0,254.0,244,1761,22.6,0.557,0.037,0.269,7.7,23.4,15.8,22.5,2.3,1.2,13.8,25.2,6.8,5.7,12.5,0.205,4.1,3.3,7.5,7.0,0.0,20.0,406.0,690.0,0.588
85,PHI,1982,Julius Erving,SF,31.0,81,81.0,2789.0,780,1428,0.546,3.0,11.0,0.273,777,1417,0.548,0.547,411,539,0.763,220.0,337.0,557.0,319,161.0,141.0,214.0,229,1974,25.9,0.593,0.008,0.377,9.3,13.0,11.2,18.1,2.8,2.9,11.4,28.1,8.7,4.6,13.3,0.229,5.6,2.1,7.8,6.9,0.0,1.0,203.0,690.0,0.294
94,SAS,1982,George Gervin,SG,29.0,79,79.0,2817.0,993,1987,0.500,10.0,36.0,0.278,983,1951,0.504,0.502,555,642,0.864,138.0,254.0,392.0,187,77.0,45.0,210.0,215,2551,24.2,0.562,0.018,0.323,5.2,9.7,7.5,11.4,1.3,0.9,8.5,35.0,8.7,2.0,10.7,0.182,4.6,-2.9,1.7,2.6,0.0,1.0,110.0,690.0,0.159
136,LAL,1982,Magic Johnson,SG,22.0,78,77.0,2991.0,556,1036,0.537,6.0,29.0,0.207,550,1007,0.546,0.540,329,433,0.760,252.0,499.0,751.0,743,208.0,34.0,286.0,223,1447,22.9,0.590,0.028,0.418,9.5,17.5,13.7,31.0,3.2,0.6,18.9,20.1,8.1,4.8,12.9,0.207,5.1,3.2,8.3,7.8,0.0,0.0,67.0,690.0,0.097
184,HOU,1982,Moses Malone,C,26.0,81,81.0,3398.0,945,1822,0.519,0.0,6.0,0.000,945,1816,0.520,0.519,630,827,0.762,558.0,630.0,1188.0,142,76.0,125.0,294.0,208,2520,26.8,0.576,0.003,0.454,17.5,21.2,19.3,6.9,1.1,2.1,11.9,29.9,11.7,3.7,15.4,0.218,5.4,-1.1,4.3,5.4,1.0,40.0,507.0,690.0,0.735
202,MIL,1982,Sidney Moncrief,SG,24.0,80,80.0,2980.0,556,1063,0.523,1.0,14.0,0.071,555,1049,0.529,0.524,468,573,0.817,221.0,313.0,534.0,382,138.0,22.0,208.0,206,1581,20.1,0.601,0.013,0.539,8.8,11.6,10.3,18.1,2.2,0.4,13.7,21.0,8.9,4.5,13.4,0.216,4.2,1.3,5.5,5.6,0.0,1.0,101.0,690.0,0.146
218,BOS,1982,Robert Parish,C,28.0,80,78.0,2534.0,669,1235,0.542,0.0,0.0,0.000,669,1235,0.542,0.542,252,355,0.710,288.0,578.0,866.0,140,68.0,192.0,221.0,267,1590,22.6,0.571,0.000,0.287,12.8,24.5,18.8,8.4,1.3,4.1,13.7,25.5,5.0,5.1,10.0,0.190,1.9,2.0,3.8,3.7,0.0,4.0,131.0,690.0,0.190
253,SEA,1982,Jack Sikma,C,26.0,82,82.0,3049.0,581,1212,0.479,2.0,13.0,0.154,579,1199,0.483,0.480,447,523,0.855,223.0,815.0,1038.0,277,102.0,107.0,213.0,268,1611,20.6,0.559,0.011,0.432,8.2,28.0,18.5,13.1,1.6,1.9,12.9,22.5,5.9,6.7,12.6,0.199,1.1,2.9,4.1,4.7,0.0,0.0,33.0,690.0,0.048
299,SEA,1982,Gus Williams,PG,28.0,80,80.0,2876.0,773,1592,0.486,9.0,40.0,0.225,764,1552,0.492,0.488,320,436,0.734,92.0,152.0,244.0,549,172.0,36.0,197.0,163,1875,21.8,0.526,0.025,0.274,3.6,5.5,4.6,31.0,2.9,0.7,9.9,28.5,5.9,4.3,10.2,0.171,4.1,0.0,4.2,4.5,0.0,2.0,115.0,690.0,0.167


In [32]:
data_new.drop(["Tm","Pos"],axis = 1)[1:]

,Year,Player,Age,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,MVP,votes_first,points_won,points_max,award_share
1,1982,Alvan Adams,27.0,79,75.0,2393.0,507,1027,0.494,0.0,1.0,0.000,507,1026,0.494,0.494,182,233,0.781,138.0,448.0,586.0,356,114.0,78.0,196.0,269,1196,18.6,0.529,0.001,0.227,6.6,20.2,13.6,22.1,2.3,1.9,14.8,22.8,2.5,4.7,7.2,0.144,0.9,2.9,3.7,3.5,0.0,0.0,0.0,0.0,0.0
2,1982,Mark Aguirre,22.0,51,20.0,1468.0,381,820,0.465,25.0,71.0,0.352,356,749,0.475,0.480,168,247,0.680,89.0,160.0,249.0,164,37.0,22.0,135.0,152,955,17.3,0.514,0.087,0.301,6.7,12.9,9.7,18.6,1.2,0.9,12.7,29.8,1.0,0.8,1.9,0.061,1.8,-1.7,0.1,0.8,0.0,0.0,0.0,0.0,0.0
3,1982,Danny Ainge,22.0,53,1.0,564.0,79,221,0.357,5.0,17.0,0.294,74,204,0.363,0.369,56,65,0.862,25.0,31.0,56.0,87,37.0,3.0,53.0,86,219,10.1,0.439,0.077,0.294,5.0,5.9,5.5,19.7,3.1,0.3,17.5,21.5,-0.3,0.8,0.5,0.042,-2.6,-0.1,-2.6,-0.1,0.0,0.0,0.0,0.0,0.0
4,1982,Tiny Archibald,33.0,68,51.0,2167.0,308,652,0.472,6.0,16.0,0.375,302,636,0.475,0.477,236,316,0.747,25.0,91.0,116.0,541,52.0,3.0,178.0,131,858,14.3,0.542,0.025,0.485,1.3,4.5,2.9,31.9,1.1,0.1,18.4,17.9,3.4,1.8,5.2,0.115,0.1,-2.5,-2.5,-0.2,0.0,0.0,0.0,0.0,0.0
5,1982,Dennis Awtrey,33.0,10,3.0,121.0,5,15,0.333,0.0,0.0,0.000,5,15,0.333,0.333,5,9,0.556,7.0,7.0,14.0,8,1.0,2.0,6.0,28,15,1.6,0.396,0.000,0.600,6.5,6.4,6.5,7.5,0.4,0.9,24.0,8.5,-0.1,0.0,0.0,-0.013,-4.5,-0.4,-4.9,-0.1,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17667,2022,Thaddeus Young,33.0,52,1.0,845.0,141,272,0.518,17.0,48.0,0.354,124,224,0.554,0.550,23,49,0.469,80.0,128.0,208.0,104,54.0,18.0,52.0,81,322,17.0,0.548,0.176,0.180,9.8,16.8,13.1,17.7,3.1,2.0,15.0,17.4,0.9,1.3,2.2,0.126,0.1,2.1,2.2,0.9,0.0,0.0,0.0,0.0,0.0
17668,2022,Trae Young,23.0,76,76.0,2652.0,711,1544,0.460,233.0,610.0,0.382,478,934,0.512,0.536,500,553,0.904,50.0,234.0,284.0,737,72.0,7.0,303.0,128,2155,25.4,0.603,0.395,0.358,2.1,9.6,5.9,46.7,1.3,0.2,14.5,34.4,9.0,1.0,10.0,0.181,7.1,-2.0,5.2,4.8,0.0,0.0,0.0,0.0,0.0
17669,2022,Omer Yurtseven,23.0,56,12.0,706.0,130,247,0.526,1.0,11.0,0.091,129,236,0.547,0.528,38,61,0.623,85.0,209.0,294.0,49,17.0,20.0,41.0,84,299,17.4,0.546,0.045,0.247,13.9,33.0,23.6,11.0,1.2,3.0,13.0,19.9,0.8,1.4,2.1,0.145,-1.4,0.4,-1.0,0.2,0.0,0.0,0.0,0.0,0.0
17670,2022,Cody Zeller,29.0,27,0.0,355.0,51,90,0.567,0.0,4.0,0.000,51,86,0.593,0.567,38,49,0.776,50.0,75.0,125.0,22,8.0,6.0,19.0,56,140,17.2,0.627,0.044,0.544,14.9,24.1,19.3,9.4,1.1,1.7,14.6,15.9,0.9,0.2,1.1,0.143,-1.2,-1.0,-2.1,0.0,0.0,0.0,0.0,0.0,0.0


# Zapisanie gotowych danych

In [14]:
data_new.to_csv(r"data/cleared_data.csv")
data_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17672 entries, 0 to 17671
Columns: 54 entries, Year to award_share
dtypes: float64(41), int64(11), object(2)
memory usage: 7.4+ MB
